In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")

In [ ]:
from diffusers import DiffusionPipeline, LCMScheduler
import torch

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
lcm_lora_id = "latent-consistency/lcm-lora-sdxl"
pipe = DiffusionPipeline.from_pretrained(model_id, use_safetensors=True, variant="fp16", torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(lcm_lora_id, adapter_name="lora")

pipe.set_adapters(["lora"], adapter_weights=[1.0])
pipe.enable_model_cpu_offload()

In [ ]:
from datetime import datetime

prompt = "They were the last two people on Earth, and the first two people on Mars."
num_images = 3

timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
for i in range(num_images):
    img = pipe(
        prompt=prompt,
        num_inference_steps=8,
        guidance_scale=1.5,
    ).images[0]
    
    img.save(f"sdxl/lcm_lora_{timestamp}_{i}.png")

In [ ]:
from diffusers import AutoPipelineForText2Image, DEISMultistepScheduler
import torch

pipe = AutoPipelineForText2Image.from_pretrained('Lykon/dreamshaper-xl-lightning', torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
from datetime import datetime

prompt = "A girl smiling"
negative_prompt = "off center"
num_images = 3

timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
for i in range(num_images):
    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=4,
        guidance_scale=2,
    ).images[0]
    
    img.save(f"sdxl/ds_lightning_{timestamp}_{i}.png")